# Проект "Исследование объявлений о продаже квартир"

## План проекта
В нашем распоряжении данные сервиса недвижимости — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. 

**Задача** — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

**Ход исследования:**
- изучение данных
- предобработка данных
- создание новых столбцов
- исследовательский анализ данных

## 1. Изучение данных

In [ ]:
import pandas as pd #импортируем библиотеку pandas

In [ ]:
data = pd.read_csv('data.csv', sep="\t") #загружаем данные из файла в датафрейм

In [ ]:
data.info() #изучаем общую информацию о датафрейме

In [ ]:
import matplotlib.pyplot as plt
data.hist(figsize=(15, 20)) #строим общую гистограмму для всех числовых столбцов таблицы
plt.show()

Изучив предложенные данные мы выяснили какие столбцы есть в таблице (**total_images, last_price, total_area, first_day_exposition, rooms, ceiling_height, floors_total, living_area, floor, is_apartment, studio, open_plan, kitchen_area, balcony, locality_name, airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition**) и построили по ним предварительные гистограммы.

## 2. Предобработка данных

**Пропуски были обнаружены в следующих столбцах:**
* ceiling_height (высота потолков (м)) 
* floors_total (всего этажей в доме) 
* living_area (жилая площадь в квадратных метрах (м²))
* is_apartment (апартаменты (булев тип)) 
* kitchen_area (площадь кухни в квадратных метрах (м²))
* balcony (число балконов)
* locality_name (название населённого пункта)
* airports_nearest (расстояние до ближайшего аэропорта в метрах (м))
* cityCenters_nearest (расстояние до центра города (м))
* parks_around3000 (число парков в радиусе 3 км)
* parks_nearest (расстояние до ближайшего парка (м))
* ponds_around3000 (число водоёмов в радиусе 3 км)
* ponds_nearest (расстояние до ближайшего водоёма (м))
* days_exposition (сколько дней было размещено объявление (от публикации до снятия))

In [ ]:
for item in data.columns: #находим пропуски в столбцах
    if len(data[item].isna().unique()) != 1:
        print(f'Количество пропусков данных в столбце {item}: {len(data[data[item].isna() == True])}')
        print() 

In [ ]:
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median()) 
#заменяем пропущенные значения в столбце с высотой потолков на медианное значение

In [ ]:
data = data.dropna(subset=['floors_total']) #удаляем пропущенные данные

In [ ]:
data['living_area'] = data['living_area'].fillna(data['living_area'].median()) #заменяем пропущенные значения в столбце с жилой площадью на медианное значение

In [ ]:
data['kitchen_area'] = data['kitchen_area'].fillna(data['kitchen_area'].median()) 
#заменяем пропущенные значения в столбце с площадью кухни на медианное значение

In [ ]:
data['balcony'] = data['balcony'].fillna(0)
#заменияем пропущенные значения в столбце с количеством балконов на 0

In [ ]:
print(len(data[data['ponds_around3000'].isna()])) #анализируем пропуски в столбцах о парках и водоёмах
print(len(data[data['parks_around3000'].isna()]))
print(len(data[(data['ponds_around3000'].isna()) & (data['ponds_nearest'].isna()) & (data['parks_nearest'].isna()) & (data['parks_around3000'].isna())]))
print(len(data[data['parks_nearest'].isna()]))
print(len(data[data['ponds_nearest'].isna()]))

In [ ]:
data['parks_around3000'] = data['parks_around3000'].fillna(0) 
#заменяем пропущенные значения в столбце с количеством парков в радиусе 3 км на 0

In [ ]:
data['ponds_around3000'] = data['ponds_around3000'].fillna(0) 
#заменяем пропущенные значения в столбце с количеством водоёмов в радиусе 3 км на 0

In [ ]:
data['days_exposition'] = data['days_exposition'].fillna(0) 
#заменяем пропущенные значения в столбце с количеством дней с публикации объявления до снятия его с публикации

**Возможные причины пропусков в данных:**
* Некоторые пропуски (например, в столбцах ceiling_heist, living_area, kitchen_area) вызваны человеческим фактором - данные просто не заполнены по ошибке или по незнанию ответа. 
* Пропуски в столбце floors_total  имеют неуточнённую природу, их мы удаляем, так как их замена только усложнит задачу и запутает данные.
* Пропуски в столбцах airports_nearest и cityCenters_nearest скорее всего также не заполнены по незнанию
* Пропуски в столбце balcony предположительно анологичны ответу '0', так как количество таких ответов не аномальное и пропуск в пункте balcony логичен именно при отсутствии балкона.
* Причины пропусков в столбце locality name не объяснимы, скорее всего так же замешан человеческий фактор, однако в этом столбце пропусков крайне мало (конверсия около 0.2%), поэтому ими можно пренебречь.
* Пропуски в столбцах parks_around3000, parks_nearest, ponds_around3000, ponds_nearest взаимосвязаны. Во всех строках где есть пропуски в столбце parks_around3000, есть пропуски и в столбцах parks_nearest, ponds_around3000, ponds_nearest. Предположительно в этих районах нет парков и водоёмов вблизи, однако в этом случае всё равно ближайшие парки и водоёмы могли бы быть указаны, поэтому эта связь не совсем понятна. Принято решение заменить в столбцах parks_around3000 и ponds_around3000 пропуски на 0, а parks_nearest и ponds_nearest оставить без изменений. Также в столбцах parks_nearest и ponds_nearest есть пропуски, не зависящие от parks_around3000 и ponds_around3000, которые могли быть вызваны недостатком информации у заполняющего.
* Пропуски в столбце days_exposition кажутся системной ошибкой, так как их достаточно много (конверсия около 13%) и эта информация скорее всего рассчитывается системой, а не заполняющим. Но также можно предположить что это значит что объявление было выставлено в день выкладки, поэтому пропуски можно заменить на 0.


In [ ]:
data.dtypes #проверяем типы данных в столбцах

**Столбцы с некорректным типом данных:**
* first_day_exposition - тип данных **object**, а так как это дата публикации, тип должен быть **datetime**
* floors_total - тип данных **float64**, а так как количество этажей всегда целое число, тип должен быть **int32**
* is_apartment - тип данных **object**, а по условию это должно быть значение типа **bool**
* balcony - тип данных **float64**, а так как количество балконов всегда целое число, тип должен быть **int32**
* parks_around3000 - тип данных **float64**, а так как количество парков всегда целое число, тип должен быть **int32**
* ponds_around3000 - тип данных **float64**, а так как количество водоёмов всегда целое число, тип должен быть **int32**
* days_exposition - тип данных **float64**, а так как количество дней с момента публикации всегда целое число, тип должен быть **int32**
* Оставшиеся столбцы с типом данных **float64** переведём в тип **float32**, а столбцы с типом **int64** переведём в тип **int32**

In [ ]:
for item in ['floors_total', 'balcony', 'parks_around3000', 'ponds_around3000', 'days_exposition', 'total_images', 'rooms', 'floor']:
    data[item] = data[item].astype('int32')
    #меняем формат данных столбцов floors_total, balcony, parks_around3000, ponds_around3000, days_exposition, total_images, rooms, floor

In [ ]:
for item in ['last_price', 'total_area', 'ceiling_height', 'living_area', 'kitchen_area', 'airports_nearest', 'cityCenters_nearest', 'parks_nearest', 'ponds_nearest']:
    data[item] = data[item].astype('float32')
    #меняем формат данных столбцов last_price, total_area, ceiling_height, living_area, kitchen_area, airports_nearest, cityCenters_nearest, parks_nearest, ponds_nearest

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S') #меняем формат данных столбца first_day_exposition

In [ ]:
data['is_apartment'] = data['is_apartment'].astype('bool') #меняем формат данных столбца is_apartment

In [ ]:
data.dtypes #проверяем все ли необходимые столбцы сменили тип

In [ ]:
data['locality_name'].sort_values().dropna().unique() #изучаем уникальные значения столбца с названиями населённых пунктов

In [ ]:
len(data['locality_name'].unique()) #оцениваем количество уникальных значений этого столбца

In [ ]:
types = sorted(['поселок городского типа', 'посёлок городского типа', 'городской поселок', 'городской посёлок', 'деревня', 'коттеджный поселок', 'коттеджный посёлок', 'посёлок при железнодорожной станции', 'поселок городского типа', 'посёлок городского типа', 'посёлок станции', 'поселок станции', 'садоводческое некоммерческое товарищество', 'садовое товарищество', 'село', 'поселок', 'посёлок'], key=len, reverse=True)

for item in data['locality_name'].dropna().unique():
    for t in types:
        if t in item:
            data[data['locality_name'] == item] = data[data['locality_name'] == item].replace(item, item[len(t)+1:])

#усредняем формат данных в столбце, приводя всё к названиям городов/посёлков и т.д. кроме случаев с названиями формата 'имени ...', их оставляем без изменений

In [ ]:
data['locality_name'].unique() #оцениваем результат

In [ ]:
len(data['locality_name'].unique()) #проверяем что количество уникальных значений столбца изменилось

In [ ]:
data['total_images'].unique() #оцениваем значения в столбце total_images, 50 фото многовато, но возможно

In [ ]:
data['last_price'].unique()
#оцениваем значения в столбце last_price, двухкомнатная квартира за 12190? 763000000? дороговато
data = data[(data['last_price'] != 12190) & (data['last_price'] != 763000000)] #удаляем редкие выбивающиеся значения

In [ ]:
data['total_area'].unique()
#оцениваем значения в столбце total_area, есть странное значение 900, но оно возможно, если мы говорим о квартире в центре Москвы, тем более у нее 12 комнат
data = data[data['total_area'] != 900] #удаляем редкое выбивающееся значение. Также минимальные значения странные, но доподлинно не ясно как они появились

In [ ]:
data['rooms'].unique() 
#оцениваем значения в столбце rooms, странные значения начинаются с 7, но это тоже возможно, если обратить внимание на предыдущее вычисление
#квартир, где больше 9 комнат всего 18, что не так значительно для статистики, поэтому ими можно пренебречь
#также есть значения равные 0, что странно но этот факт не удаётся ни с чем связать
data = data.loc[data['rooms'] < 9]

In [ ]:
sorted(data['ceiling_height'].unique()) #оцениваем значения в столбце ceiling_height, значения выше 4 кажутся нереальными, потому что даже в сталинках потолки до 3,6 м, тем не менее стандарт потолков - от 2,5 м, так что можно предположить что значения выше 25 - это значения, внесённые с ошибкой
data.loc[data['ceiling_height'] >= 25, 'ceiling_height'] /= 10 #приводим значения выше 25 к более реалистичным, деля на 10

In [ ]:
data['living_area'].unique() #оцениваем значения в столбце living_area, минимальные значения странные, но их природа доподлинно не ясна


In [ ]:
data['kitchen_area'].unique() #оцениваем значения в столбце kitchen_area, 100 квадратов для кухни многовато, но если соотносить с общей площадью - это возможно

In [ ]:
data['airports_nearest'].unique() #оцениваем значения в столбце airports_nearest, находим квартиру находящуюся прямо в аэропорту
data = data.loc[data['airports_nearest'] != 0] #удаляем это значение

In [ ]:
data['cityCenters_nearest'].unique() #оцениваем значения в столбце cityCenters_nearest, аномалий не обнаружено

**Вывод об особенностях данных:**
* В столбце **total_images** аномалий не выявлено
* В столбце **last_price** было выявлено две аномалии непонятной природы, были удалены как выбивающиеся значения (двухкомнатная квартира за 12190 и квартира стоимостью 763000000)
* В столбце **total_area** было удалено краевое, слишком большое значение (квартира площадью 900 кв.м.)
* В столбце **rooms** квартир, где больше 9 комнат было всего 18, что не так значительно для статистики, поэтому мы удалили их как краевые значения
* В столбце **ceiling_height** большая часть значений кажется нереалистичной но только в части из них можно было логически предположить ошибки. Все значения выше 25 м разделили на 10 чтобы привести к более реалистичным значениям
* В столбце **living_area** минимальные значения кажутся нереалистичными но причины возникновения таких значений не ясна
* В столбце **kitchen_area** есть подозрительно высокие значения но они логично соотносятся с площадью этих квартир, поэтому значения этого столбца оставлены без изменений
* В столбце **airports_nearest** была найдена квартира с расстоянием 0 км до аэропорта, что невозможно, поэтому это значение было удалено
* В столбце **cityCenters_nearest** аномалии не обнаружены

**Вывод о предобработке данных:**
* В ходе предобработки пропуски данных были обнаружены в следующих столбцах: **ceiling_height** (высота потолков (м)) - **9195** пропусков, **floors_total** (всего этажей в доме) - **86** пропусков, **living_area** (жилая площадь в квадратных метрах (м²)) - **1903** пропуска, **is_apartment** (апартаменты (булев тип)) - **20924** пропусков, **kitchen_area** (площадь кухни в квадратных метрах (м²)) - **2278** пропусков, **balcony** (число балконов) - **11519** пропусков, **locality_name** (название населённого пункта) - **49** пропусков, **airports_nearest** (расстояние до ближайшего аэропорта в метрах (м)) - **5542** пропуска, **cityCenters_nearest** (расстояние до центра города (м)) - **5519** пропусков, **parks_around3000** (число парков в радиусе 3 км) - **5518** пропусков, **parks_nearest** (расстояние до ближайшего парка (м)) - **15620** пропусков, **ponds_around3000** (число водоёмов в радиусе 3 км) - **5518** пропусков, **ponds_nearest** (расстояние до ближайшего водоёма (м)) - **14589** пропусков, **days_exposition** (сколько дней было размещено объявление (от публикации до снятия)) - **3181** пропусков.
* Некоторые пропуски (например, в столбцах **ceiling_heist, living_area, kitchen_area**) вызваны человеческим фактором - данные просто не заполнены по ошибке или по незнанию ответа. Пропуски в столбце **floors_total**  имеют неуточнённую природу, их мы удаляем, так как их замена только усложнит задачу и запутает данные. Пропуски в столбцах **airports_nearest и cityCenters_nearest** скорее всего также не заполнены по незнанию. Пропуски в столбце **balcony** предположительно анологичны ответу '0', так как количество таких ответов не аномальное и пропуск в пункте **balcony** логичен именно при отсутствии балкона. Причины пропусков в столбце **locality_name** не объяснимы, скорее всего так же замешан человеческий фактор, однако в этом столбце пропусков крайне мало (конверсия около 0.2%), поэтому ими можно пренебречь. Пропуски в столбцах **parks_around3000, parks_nearest, ponds_around3000, ponds_nearest** взаимосвязаны. Во всех строках где есть пропуски в столбце **parks_around3000**, есть пропуски и в столбцах **parks_nearest, ponds_around3000, ponds_nearest**. Предположительно в этих районах нет парков и водоёмов вблизи, однако в этом случае всё равно ближайшие парки и водоёмы могли бы быть указаны, поэтому эта связь не совсем понятна. Принято решение заменить в столбцах **parks_around3000 и ponds_around3000** пропуски на 0, а **parks_nearest и ponds_nearest** оставить без изменений. Также в столбцах **parks_nearest и ponds_nearest** есть пропуски, не зависящие от **parks_around3000 и ponds_around3000**, которые могли быть вызваны недостатком информации у заполняющего. Пропуски в столбце **days_exposition** кажутся системной ошибкой, так как их достаточно много (конверсия около 13%) и эта информация скорее всего рассчитывается системой, а не заполняющим. Но также можно предположить что это значит что объявление было выставлено в день выкладки, поэтому пропуски можно заменить на 0.
* Некорректный тип данных был обнаружен в следующих столбцах: **first_day_exposition** - тип данных **object**, а так как это дата публикации, тип должен быть **datetime**, **floors_total** - тип данных **float64**, а так как количество этажей всегда целое число, тип должен быть **int32**, **is_apartment** - тип данных **object**, а по условию это должно быть значение типа **bool**, **balcony** - тип данных **float64**, а так как количество балконов всегда целое число, тип должен быть **int32**, **parks_around3000** - тип данных **float64**, а так как количество парков всегда целое число, тип должен быть **int32**, **ponds_around3000** - тип данных **float64**, а так как количество водоёмов всегда целое число, тип должен быть **int32**, **days_exposition** - тип данных **float64**, а так как количество дней с момента публикации всегда целое число, тип должен быть **int32**, оставшиеся столбцы с типом данных **float64** переведём в тип **float32**, а столбцы с типом **int64** переведём в тип **int32**.
* В столбце **last_price** было выявлено две аномалии непонятной природы, были удалены как выбивающиеся значения (двухкомнатная квартира за 12190 и квартира стоимостью 763000000), в столбце **total_area** было удалено краевое, слишком большое значение (квартира площадью 900 кв.м.), в столбце **rooms** квартир, где больше 9 комнат было всего 18, что не так значительно для статистики, поэтому мы удалили их как краевые значения, в столбце **ceiling_height** большая часть значений кажется нереалистичной но только в части из них можно было логически предположить ошибки. Все значения выше 25 м разделили на 10 чтобы привести к более реалистичным значениям. В столбце **living_area** минимальные значения кажутся нереалистичными но причины возникновения таких значений не ясна, в столбце **kitchen_area** есть подозрительно высокие значения но они логично соотносятся с площадью этих квартир, поэтому значения этого столбца оставлены без изменений, в столбце **airports_nearest** была найдена квартира с расстоянием 0 км до аэропорта, что невозможно, поэтому это значение было удалено.

## 3. Считаем и добавляем в таблицу новые столбцы

In [ ]:
data['cost_per_square_meter'] = data['last_price'] / data['total_area'] #создаём столбец с ценой за квадратный метр

In [ ]:
from datetime import datetime
weekday = []
for item in data['first_day_exposition']:
    weekday.append(datetime.weekday(item))
data['weekday'] = weekday #формируем столбец с днями недели выкладки объявления


In [ ]:
data['month'] = pd.DatetimeIndex(data['first_day_exposition']).month #добавляем столбец с месяцем выкладки объявления

In [ ]:
data['year'] = pd.DatetimeIndex(data['first_day_exposition']).year #добавляем столбец с годом выкладки объявления

In [ ]:
data['floor_type'] = ['другой' for _ in range (len(data))] #создаём столбец с типами этажей
data.loc[data['floor'] == data['floors_total'], 'floor_type'] = 'последний' #если этаж и количество этажей в доме совпадает
data.loc[data['floor'] == 1, 'floor_type'] = 'первый' #если значение этажа равно 1

In [ ]:
data['city_center_km'] = [0 for i in range (len(data))] #создаём столбец с расстояниями до центра города в километрах
for item in data['cityCenters_nearest']:
    try:
        data.loc[data['cityCenters_nearest'] == item, 'city_center_km'] = round(item / 1000)
    except:
        continue

## 4. Исследовательский анализ данных

In [ ]:
data['total_area'].plot(kind='hist', title='total_area', range=(0, 350), bins=10) 
plt.show()
#построение гистограммы по столбцу total_area

In [ ]:
data['living_area'].plot(kind='hist', title='living_area', range=(0, 230), bins=10)
plt.show()
#построение гистограммы по столбцу living_area

In [ ]:
data['kitchen_area'].plot(kind='hist', title='kitchen_area', range=(0, 60), bins=10)
plt.show()
#построение гистограммы по столбцу kitchen_area

In [ ]:
data['rooms'].plot(kind='hist', title='rooms', range=(0,10), bins=10) 
plt.show()
#построение гистограммы по столбцу rooms

In [ ]:
data['last_price'].plot(kind='hist', title='last_price', range=(0, 50000000), bins=10) 
plt.show()
#построение гистограммы по столбцу last_price

In [ ]:
data['ceiling_height'].plot(kind='hist', title='ceiling_height', range=(0, 10), bins=10) 
plt.show()
#построение гистограммы по столбцу ceiling_height

In [ ]:
data['floor'].plot(kind='hist', title='floor', range=(0, 50), bins=10) #построение гистограммы по столбцу floor
plt.show()

In [ ]:
data['floor_type'].hist(bins=10) #построение гистограммы по столбцу floor_type
plt.show()

In [ ]:
data['floors_total'].plot(kind='hist', title='floors_total', range=(0, 50), bins=10) #построение гистограммы по столбцу floor_total
plt.show()

In [ ]:
data['cityCenters_nearest'].plot(kind='hist', title='cityCenters_nearest', range=(0, 60000), bins=10) #построение гистограммы по столбцу cityCenters_nearest
plt.show()

In [ ]:
data['airports_nearest'].plot(kind='hist', title='airports_nearest', bins=10) #построение гистограммы по столбцу airports_nearest
plt.show()

In [ ]:
data['parks_nearest'].plot(kind='hist', title='parks_nearest', bins=10) #построение гистограммы по столбцу parks_nearest
plt.show()

In [ ]:
data['weekday'].plot(kind='hist', title='weekday', bins=31) #построение гистограммы по столбцу weekday
plt.show()

In [ ]:
data['month'].plot(kind='hist', title='month', bins=12) #построение гистограммы по столбцу month
plt.show()

**Выводы по данным гистограмм:**
* Из гистограммы по столбцу **total_area** можно заметить что самая частотная общая площадь - в районе 50 кв.м.
* Из гистограммы по столбцу **living_area** можно заметить что у большей части квартир жилая площадь менее 50 кв.м., самая частая площадь - от 25 до 40 кв.м.
* Из гистограммы по столбцу **kitchen_area** можно заметить что большая часть кухонь имеет площадь в районе 10 кв.м.
* Из гистограммы по столбцу **last_price** можно заметить что большая часть квартир имеет цену ниже 4900000
* Из гистограммы по столбцу **ceiling_height** можно заметить что высота потолка зачастую находится в районе 2-3 м
* Из гистограммы по столбцу **floor** можно заметить что чаще всего встречаются квартиры до 10 этажа, самые частые - первые 4 этажа
* Из гистограммы по столбцу **floor_type** можно заметить что число квартир на верхнем и нижнем этажах почти одинаково, но на последнем - немного больше. Ожидаемо количество остальных этажей больше всего.
* Из гистограммы по столбцу **floors_total** можно заметить что чаще всего продавались квартиры в домах с 5-10 этажами, следом за ними - дома с 10-20 этажами, за ними - с 20-30.
* Из гистограммы по столбцу **cityCenters_nearest** можно заметить что самое частое расстояние до центра 12000-18000 м
* Из гистограммы по столбцу **airports_nearest** можно заметить что самое частое расстояние до аэропорта - 15000-35000 м
* Из гистограммы по столбцу **parks_nearest** можно заметить что самое частое расстояние до ближайшего парка - в районе 500 м или меньше
* Из гистограммы по столбцу **weekday** можно заметить что самые частые дни для выкладки объявления - вторник и четверг, самые редкие - выходные.
* Из гистограммы по столбцу **month** можно заметить что наибольшее число объявлений приходится на период с февраля по апрель, за ним идёт период с сентября по ноябрь. Самые резкие провалы в количестве объявлений приходятся на май и декабрь.

In [ ]:
data['days_exposition'].plot(kind='hist', title='days_exposition_under_100', bins=100, range=(0, 100)) 
plt.show()
#построение гистограммы по столбцу days_exposition

In [ ]:
data['days_exposition'].plot(kind='hist', title='days_exposition_under_1300', bins=100, range=(1300, 1600)) 
plt.show()
#построение гистограммы по столбцу days_exposition

In [ ]:
data.pivot_table(index='first_day_exposition', values='days_exposition', aggfunc=['mean', 'median'])
#построение сводной таблицы по дням размещения объявления

**Оценка столбца days_exposition:**
По данным столбца можно понять что большая часть продаётся до 200 дней, чаще всего - до 75. Выбросы находятся в районе 1000-1400 дней, это редкие и необычно долгие продажи. Наибольшее число выбросов - в районе 1310, 1350, то есть около 3,5 лет, что неправдоподобно много. Максимальное время продажи - 1580, то есть в районе 4,2 лет. Предполагаю что это объявления, которые продались через другой сервис или через Яндекс Недвижимость но автор объявления забыл его закрыть. Большое количество нулей вызвано тем что ранее мы заменяли пропуски на нули.

In [ ]:
data.pivot_table(index='total_area', values='last_price') #создаём свобдную таблицу зависимости цены от общей площади

In [ ]:
data.plot(x='total_area', y='last_price', kind='scatter', title='total_area', alpha=0.05) 
plt.show()
#строим график зависимости цены от общей площади

In [ ]:
data.pivot_table(index='living_area', values='last_price') #создаём свобдную таблицу зависимости цены от жилой площади

In [ ]:
data.plot(x='living_area', y='last_price', kind='scatter', title='living_area', alpha=0.08) 
plt.show()
#строим график зависимости цены от жилой площади

In [ ]:
data.pivot_table(index='kitchen_area', values='last_price') #создаём свобдную таблицу зависимости цены от площади кухни

In [ ]:
data.plot(x='kitchen_area', y='last_price', kind='scatter', title='kitchen_area', alpha=0.08) 
plt.show()
#строим график зависимости цены от площади кухни

In [ ]:
data.pivot_table(index='rooms', values='last_price') #создаём свобдную таблицу зависимости цены от количества комнат

In [ ]:
plt.bar(data['rooms'], data['last_price'])
plt.show()

In [ ]:
data.pivot_table(index='floor_type', values='last_price') #создаём свобдную таблицу зависимости цены от типа этажа

In [ ]:
plt.bar(data['floor_type'], data['last_price'])
plt.show()
#строим график зависимости цены от типа этажа

In [ ]:
data.pivot_table(index='weekday', values='last_price') #создаём свобдную таблицу зависимости цены от дня недели

In [ ]:
plt.bar(data['weekday'], data['last_price'])
plt.show()
#строим график зависимости цены от дня недели

In [ ]:
data.pivot_table(index='month', values='last_price') #создаём свобдную таблицу зависимости цены от месяца

In [ ]:
plt.bar(data['month'], data['last_price'])
plt.show() #строим график зависимости цены от месяца

In [ ]:
data.pivot_table(index='year', values='last_price') #создаём свобдную таблицу зависимости цены от года

In [ ]:
plt.bar(data['year'], data['last_price'])
plt.show() #строим график зависимости цены от года

**Зависимости цены от других показателей:**
* По графику можно заметить что зависимость цены от общей площади прямая, если не обращать внимания на редкие выбросы. То есть чем больше площадь - тем дороже квартира.
* Подобная же ситуация наблюдается с жилой площадью и площадью кухни, так как это взаимосвязанные вещи, однако на этих графиках замечены выбросы на значениях 25 кв.м. для жилой площади и 10 кв.м. для площади кухни.
* Связь между числом комнат и ценой также прямая. График выглядит параболически но это скорее всего связно с недостатком данных для квартир где более 5 комнат. 
* Зависимость цены от этажей такова - самые дешевые квартиры - на первом этаже. Цены последних и остальных этажей примерно одинакова, но по последним этажам меньше данных.
* По графику зависимости цены от дня недели можно заметить что самый дорогой день - суббота, а самый дешёвый - воскресенье.
* По графику зависимости цены от месяца можно заметить что самый дорогой месяц - февраль, а самый дешёвый - июнь.
* По графику зависимости цены от года можно заметить что дешевле всего квартиры продавались в 2014 году, дороже всего - в 2016.

In [ ]:
data.pivot_table(index='locality_name', values='cost_per_square_meter') #строим сводную таблицу по стоимости квадратного метра в каждом населённом пункте

In [ ]:
data.groupby('locality_name')['total_images'].count().sort_values(ascending=False).head(10) #находим 10 населённых пунктов с наибольшим числом объявлений

In [ ]:
special_cities = ["Санкт-Петербург", "Мурино", "Кудрово", "Шушары", "Всеволожск", "Пушкин", "Колпино", "Парголово", "Выборг"]
special_cities_data = data.query('locality_name in @special_cities')
special_cities_data.groupby('locality_name').agg({'cost_per_square_meter':['mean']}) #считаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений

Минимальная средняя цена за квадратный метр в **Выборге - 58141.909153**, максимальная - в **Санкт-Петербурге - 114725.667939**

In [ ]:
d = data.pivot_table(index='city_center_km', values='last_price', aggfunc='mean') #считаем среднюю цену для каждого километра от центра

In [ ]:
d.plot(y='last_price', style='o-', figsize=(25, 5))
plt.show()

Зависимость цены от расстояния до центра города **обратная** - чем меньше расстояние до центра, тем дороже квартира, однако зависимость не сильная, коэффициент корреляции -23%. Есть пиковые значения - самые дорогие квартиры - в километре от центра, самые дешёвые - в 60 км. Есть неожиданный спад на уровне 3 км от центра, эти квартиры дешевле чем те, от которых 7-8 км до центра. Также выбросы есть на уровне 44 км и 55 км. 

### Общий вывод

**В ходе исследования были сделаны следующие выводы:**
* В процессе предобработки были найдены пропуски в следующих столбцах: **ceiling_height** (высота потолков (м)) - **9195 пропусков**, **floors_total** (всего этажей в доме) - **86 пропусков**, **living_area** (жилая площадь в квадратных метрах (м²)) - **1903 пропуска**, **is_apartment** (апартаменты (булев тип)) - **20924 пропуска**, **kitchen_area** (площадь кухни в квадратных метрах (м²)) - **2278 пропусков**, **balcony** (число балконов) - **11519 пропусков**, **locality_name** (название населённого пункта) - **49 пропусков**, **airports_nearest** (расстояние до ближайшего аэропорта в метрах (м)) - **5542 пропуска**, **cityCenters_nearest** (расстояние до центра города (м)) - **5519 пропусков**, **parks_around3000** (число парков в радиусе 3 км) - **5518 пропусков**, **parks_nearest** (расстояние до ближайшего парка (м)) - **15620 пропусков**, **ponds_around3000** (число водоёмов в радиусе 3 км) - **5518 пропусков**, **ponds_nearest** (расстояние до ближайшего водоёма (м)) - **14589 пропусков**, **days_exposition** (сколько дней было размещено объявление (от публикации до снятия)) - **3181 пропуск**.
* Некоторые пропуски (например, в столбцах **ceiling_heist, floors_total, living_area, kitchen_area**) вызваны человеческим фактором - данные просто не заполнены по ошибке или по незнанию ответа. Пропуски в столбцах **airports_nearest** и **cityCenters_nearest** скорее всего также не заполнены по незнанию. Пропуски в столбце **balcony** предположительно анологичны ответу '0', так как количество таких ответов не аномальное и пропуск в пункте balcony логичен именно при отсутствии балкона. Причины пропусков в столбце **locality name** не объяснимы, скорее всего так же замешан человеческий фактор, однако в этом столбце пропусков крайне мало (конверсия около 0.2%), поэтому ими можно пренебречь. Пропуски в столбцах **parks_around3000, parks_nearest, ponds_around3000, ponds_nearest** взаимосвязаны. Во всех строках где есть пропуски в столбце **parks_around3000**, есть пропуски и в столбцах **parks_nearest, ponds_around3000, ponds_nearest**. Предположительно в этих районах нет парков и водоёмов вблизи, однако в этом случае всё равно ближайшие парки и водоёмы могли бы быть указаны, поэтому эта связь не совсем понятна. Принято решение заменить в столбцах **parks_around3000** и **ponds_around3000** пропуски на 0, а **parks_nearest** и **ponds_nearest** оставить без изменений. Также в столбцах **parks_nearest** и **ponds_nearest** есть пропуски, не зависящие от **parks_around3000** и **ponds_around3000**, которые могли быть вызваны недостатком информации у заполняющего. Пропуски в столбце **days_exposition** кажутся системной ошибкой, так как их достаточно много (конверсия около 13%) и эта информация скорее всего рассчитывается системой, а не заполняющим. Но также можно предположить что это значит что объявление было выставлено в день выкладки, поэтому пропуски можно заменить на 0.
* Были также обнаружены столбцы с некорректными типами данных - **first_day_exposition** - тип данных **object**, а так как это дата публикации, тип должен быть **datetime**, **floors_total** - тип данных **float64**, а так как количество этажей всегда целое число, тип должен быть **int**, **is_apartment** - тип данных **object**, а по условию это должно быть значение типа **bool**, **balcony** - тип данных **float64**, а так как количество балконов всегда целое число, тип должен быть **int**, **parks_around3000** - тип данных **float64**, а так как количество парков всегда целое число, тип должен быть **int**, **ponds_around3000** - тип данных **float64**, а так как количество водоёмов всегда целое число, тип должен быть **int**, **days_exposition** - тип данных **float64**, а так как количество дней с момента публикации всегда целое число, тип должен быть **int**. Типы данных были заменены на корректные.
* В столбце с названиями населённых пунктов были обнаружены неявные дубликаты, например, «поселок Рябово» и «поселок городского типа Рябово», «поселок Тельмана» и «посёлок Тельмана» — это обозначения одних и тех же населённых пунктов. Такие дубликаты были устранены.
* Столбцы были изучены на аномалии и вот что было выявлено: в столбце **total_images** аномалий не выявлено; в столбце **last_price** было выявлено две аномалии непонятной природы, были удалены как выбивающиеся значения (двухкомнатная квартира за 12190 и квартира стоимостью 763000000); в столбце **total_area** было удалено краевое, слишком большое значение (квартира площадью 900 кв.м.); в столбце **rooms** квартир, где больше 9 комнат было всего 18, что не так значительно для статистики, поэтому мы удалили их как краевые значения; в столбце **ceiling_height** большая часть значений кажется нереалистичной но только в части из них можно было логически предположить ошибки. Все значения выше 25 м разделили на 10 чтобы привести к более реалистичным значениям; в столбце **living_area** минимальные значения кажутся нереалистичными но причины возникновения таких значений не ясна; в столбце **kitchen_area** есть подозрительно высокие значения но они логично соотносятся с площадью этих квартир, поэтому значения этого столбца оставлены без изменений; в столбце **airports_nearest** была найдена квартира с расстоянием 0 км до аэропорта, что невозможно, поэтому это значение было удалено; в столбце **cityCenters_nearest** аномалии не обнаружены.
* В таблицу были добавлены следующие столбцы: **cost_per_square_meter** - цена одного квадратного метра, **weekday** - день недели публикации объявления (0 — понедельник, 1 — вторник и так далее), **month** - месяц публикации объявления, **year** - год публикации объявления, **floor_type** - тип этажа квартиры (значения — «первый», «последний», «другой»), **city_center_km** - расстояние до центра города в километрах (переведите из м в км и округлите до целых значений).
* На этапе проведения исследовательского анализа данных были изучены следующие параметры объектов - общая площадь, жилая площадь, площадь кухни, цена объекта, количество комнат, высота потолков, этаж квартиры, тип этажа квартиры («первый», «последний», «другой»), общее количество этажей в доме, расстояние до центра города в метрах, расстояние до ближайшего аэропорта, расстояние до ближайшего парка, день и месяц публикации объявления. Были построены гистограммы для каждого параметра и сделаны следующие выводы: из гистограммы по столбцу **total_area** можно заметить что самая частотная общая площадь - в районе 50 кв.м.; из гистограммы по столбцу **living_area** можно заметить что у большей части квартир жилая площадь менее 50 кв.м., самая частая площадь - от 25 до 40 кв.м.; из гистограммы по столбцу **kitchen_area** можно заметить что большая часть кухонь имеет площадь в районе 10 кв.м.; из гистограммы по столбцу **last_price** можно заметить что большая часть квартир имеет цену ниже 4900000; из гистограммы по столбцу **ceiling_height** можно заметить что высота потолка зачастую находится в районе 2-3 м; из гистограммы по столбцу **floor** можно заметить что чаще всего встречаются квартиры до 10 этажа, самые частые - первые 4 этажа; из гистограммы по столбцу **floor_type** можно заметить что число квартир на верхнем и нижнем этажах почти одинаково, но на последнем - немного больше. Ожидаемо количество остальных этажей больше всего.; из гистограммы по столбцу **floors_total** можно заметить что чаще всего продавались квартиры в домах с 5-10 этажами, следом за ними - дома с 10-20 этажами, за ними - с 20-30.; из гистограммы по столбцу **cityCenters_nearest** можно заметить что самое частое расстояние до центра 12000-18000 м; из гистограммы по столбцу **airports_nearest** можно заметить что самое частое расстояние до аэропорта - 15000-35000 м; из гистограммы по столбцу **parks_nearest** можно заметить что самое частое расстояние до ближайшего парка - в районе 500 м или меньше; из гистограммы по столбцу **weekday** можно заметить что самые частые дни для выкладки объявления - вторник и четверг, самые редкие - выходные.; из гистограммы по столбцу **month** можно заметить что наибольшее число объявлений приходится на период с февраля по апрель, за ним идёт период с сентября по ноябрь. Самые резкие провалы в количестве объявлений приходятся на май и декабрь.
* По данным столбца **days_exposition** можно понять что большая часть объектов продаётся до 200 дней, чаще всего - до 75. Выбросы находятся в районе 1000-1400 дней, это редкие и необычно долгие продажи.
* Была изучена зависимость цены объекта от разных факторов и вот к каким выводам удалось прийти: по графику можно заметить что зависимость цены от общей площади прямая, если не обращать внимания на редкие выбросы. То есть чем больше площадь - тем дороже квартира. Подобная же ситуация наблюдается с жилой площадью и площадью кухни, так как это взаимосвязанные вещи, однако на этих графиках замечены выбросы на значениях 25 кв.м. для жилой площади и 10 кв.м. для площади кухни. Связь между числом комнат и ценой также прямая. График выглядит параболически но это скорее всего связно с недостатком данных для квартир где более 5 комнат. Зависимость цены от этажей такова - самые дешевые квартиры - на первом этаже. Цены последних и остальных этажей примерно одинакова, но по последним этажам меньше данных. По графику зависимости цены от дня недели можно заметить что самый дорогой день - суббота, а самый дешёвый - воскресенье. По графику зависимости цены от месяца можно заметить что самый дорогой месяц - февраль, а самый дешёвый - июнь. По графику зависимости цены от года можно заметить что дешевле всего квартиры продавались в 2014 году, дороже всего - в 2016.
* Были выявлены 10 городов с наибольшим числом объявлений - **Санкт-Петербург, Мурино, Кудрово, Шушары, Всеволожск, Пушкин, Колпино, Парголово, Выборг** и высчитана средняя цена квадратного метра в этих городах. Минимальная средняя цена за квадратный метр в **Выборге - 58141.909153**, максимальная - в **Санкт-Петербурге - 114725.667939**
* Была изучена зависимость стоимости квартиры в Санкт-Петербурге от расстояния от квартиры до центра и построен график. Зависимость цены от расстояния до центра города **обратная** - чем меньше расстояние до центра, тем дороже квартира, однако зависимость не сильная, коэффициент корреляции -23%. Есть пиковые значения - самые дорогие квартиры - в километре от центра, самые дешёвые - в 60 км. Есть неожиданный спад на уровне 3 км от центра, эти квартиры дешевле чем те, от которых 7-8 км до центра. Также выбросы есть на уровне 44 км и 55 км. 
